In [33]:
%env SITE=grenoble

env: SITE=grenoble


In [34]:
%env CHANNEL=19

env: CHANNEL=19


In [35]:
%env PAN_ID=0xBEEF

env: PAN_ID=0xBEEF


In [36]:
!iotlab-profile addm3 -n consumptionm3 -power -voltage -current  -period 8244 -avg 4

HTTP Error 500: 
	{"code":500,"message":"Profile consumptionm3 already exist"}


In [ ]:
!iotlab-profile adda8 -n consumptiona8 -power -voltage -current -period 8244 -avg 4

In [38]:
!iotlab-experiment submit -n "assignment2-m3" -d 120 -l 1,archi=a8:at86rf231+site=$SITE,,consumptiona8 -l 3,archi=m3:at86rf231+site=$SITE,,consumptionm3

{
    "id": 271957
}


In [39]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

Waiting that experiment 271957 gets in state Running
"Running"


In [40]:
!iotlab-experiment --jmespath="items[*].network_address | sort(@)" get --nodes

[
    "a8-106.grenoble.iot-lab.info",
    "m3-108.grenoble.iot-lab.info",
    "m3-109.grenoble.iot-lab.info",
    "m3-110.grenoble.iot-lab.info"
]


### Deploy public IPv6 network

1. Compile the RIOT gnrc_border router example

A border router (BR) is a routing device to connect a wireless sensor network to the Internet based on IPv6 technology. Choose one node of your experiment which play the role of BR. Compile and flash BR firmware to this node. Replace `<id>` with the right value:

In [ ]:
ssh $IOTLAB_LOGIN@saclay.iot-lab.info

In [ ]:
!mkdir -p ~/riot
!cd ~/riot
!rm -rf RIOT
!git clone https://github.com/RIOT-OS/RIOT.git -b 2020.10-branch
!cd RIOT    
!source /opt/riot.source

In [ ]:
!make DEFAULT_CHANNEL=19 BOARD=iotlab-m3 -C examples/gnrc_border_router clean all
!iotlab-node --flash examples/gnrc_border_router/bin/iotlab-m3/gnrc_border_router.elf -l saclay,m3,10
!sudo ethos_uhcpd.py m3-10 tap0 2001:660:3207:04c1::1/64

In [ ]:
!cp $BIN_DIR_BR/gnrc_border_router.elf
!iotlab-node --flash examples/gnrc_border_router/bin/iotlab-m3/gnrc_border_router.elf -l saclay,m3,6
!sudo ethos_uhcpd.py m3-1 tap0 2001:660:3207:04c1::1/64

### Launch the commands from the first terminal

From here, open a Jupyter terminal (use `File > New > Terminal`).

Connect to the SSH frontend and replace `<site>` with the right value.

In [ ]:
ssh root@node-a8-<id>

In [ ]:
iotlab_flash A8/gnrc_border_router.elf

In [ ]:
cd A8/riot/RIOT/dist/tools/uhcpd

In [ ]:
make clean all

In [ ]:
cd ../ethos

In [ ]:
make clean all

In [ ]:
reset_a8_m3

In [ ]:
./start_network.sh /dev/ttyA8_M3 tap0 2001:660:3207:401::/64 500000

In [ ]:
printenv (know the prefix)

### Launch the commands from the second terminal

From here, open a Jupyter terminal (use `File > New > Terminal`).

Connect to the SSH frontend and replace `<site>` with the right value.

In [ ]:
ssh root@node-a8-<id>

In [ ]:
scp $IOTLAB_LOGIN@saclay.iot-lab.info:hub/user-redirect/lab/tree/riot/networking/assignment2/config.conf config.con

In [ ]:
ip -6 -o addr show eth0

In [ ]:
This global address must be noted and used later to connect to the MQTT Broker

In [ ]:
broker_mqtts config.conf

In [ ]:
%env BR_ID = 102
%env APP_DIR_EMCUTE = ../../RIOT/examples/emcute_mqttsn
%env BIN_DIR_EMCUTE = ../../RIOT/examples/emcute_mqttsn/bin/iotlab-a8-m3
!source /opt/riot.source
!make -C $APP_DIR_EMCUTE ETHOS_BAUDRATE=500000 BOARD=iotlab-a8-m3 DEFAULT_CHANNEL=$CHANNEL DEFAULT_PAN_ID=$PAN_ID clean all
!cp $BIN_DIR_EMCUTE/emcute_mqttsn.elf  ~/A8/

### Launch the commands from the third  terminal

From here, open a Jupyter terminal (use `File > New > Terminal`).

Connect to the SSH frontend and replace `<site>` with the right value.

In [ ]:
ssh root@node-a8-<id>

In [ ]:
iotlab_flash A8/emcute_mqttsn.elf

In [ ]:
reset_a8_m3

6. Open a Jupyter terminal (use `File > New > Terminal`) and open the serial link.

In [ ]:
miniterm.py /dev/ttyA8_M3 500000 

Interact with the shell to get the command output:

con (ip del broker) 1885

In [ ]:
sub topic

### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [ ]:
!iotlab-experiment stop